In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
l1 = list(np.arange(1,54))
l2 = [56,57,58,62,63,64,65,66,69,70,73,75,77,78,80,81,85,87,89,90,91,92,96,97,98,99,101,102,103]
areas = l1+l2

In [ ]:
features = pd.read_csv("Datasets/dataset_features_users_POIs.csv")
# targets = targets.drop(columns=["Unnamed: 0","Unnamed: 0.1", "qtd_users"])
features = features.drop(columns=["Unnamed: 0"])
features.head()

In [ ]:
targets = pd.read_csv("Datasets/dataset_soma_ocorr_2022.csv")

# targets = targets.drop(columns=["Unnamed: 0","Unnamed: 0.1", "qtd_users"])
targets = targets.drop(columns=["Unnamed: 0"])
# targets.head()

In [ ]:
targets["area"] = areas

In [ ]:
targets = targets[["area","total_homicidios", "total_estupro", "total_les_corporal", "total_roubo2", "total_furt"]]

In [ ]:
# Definindo a semente aleatória como 42
np.random.seed(42)

# separar as features
X = features.values

scaler = StandardScaler()
X = scaler.fit_transform(X)

# criar um objeto do modelo Random Forest Regressor
model = KNeighborsRegressor(n_neighbors=5)

# criar um objeto LeaveOneOut
loo = LeaveOneOut()

#matriz de metricas RAE, MAE e R2
metricas = pd.DataFrame(columns=['Ocorrencia','RAE','MAE','R2'])
ocorrencias = []
media_rae = []
media_mae =[]
media_r2 = []


for i in targets.columns:
  list_y_test = []
  list_y_pred = []
  # separa a variável target
  y = targets[i].values


  # percorrer cada amostra do conjunto de dados
  for train_index, test_index in loo.split(X):

      # separar os conjuntos de treinamento e teste
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]

      # treinar o modelo
      model.fit(X_train, y_train)

      #adiciona a importancia das features para predição da target i
      #importances[i] = model.feature_importances_

      # fazer a previsão usando o conjunto de teste
      y_pred = model.predict(X_test)

      list_y_test.append(y_test)
      list_y_pred.append(y_pred)


  # calcular o erro relativo absoluto (RAE) e adiciona na lista
  media_rae.append(mean_absolute_error(list_y_test, list_y_pred) / np.mean(list_y_test))
  # calcular o erro absoluto medio (MAE) e adiciona na lista
  media_mae.append(mean_absolute_error(list_y_test, list_y_pred))
  # calcular R2 e adiciona na lista
  media_r2.append(r2_score(list_y_test,list_y_pred))
  #tipo de ocorrencia
  ocorrencias.append(i)


metricas["Ocorrencia"] = ocorrencias
metricas["RAE"] = media_rae
metricas["MAE"] = media_mae
metricas["R2"] = media_r2

In [ ]:
metricas